In [1]:
from wand.image import Image as WImage
from PIL import Image as PI
import pyocr
import pyocr.builders
import io
import os
from pathlib import Path
import django
from django.db import transaction

path = Path.joinpath(Path.cwd(), 'pdf')
os.environ['DJANGO_SETTINGS_MODULE'] = 'courrier.settings'
django.setup()

from ocr.models import OcrDocument, OcrPage

In [2]:
tool = pyocr.get_available_tools()[0]
lang = 'fra'

Unsupported version [0.0.0]


In [3]:
def extract_text(file):
    """Extract the text from the file path to a list of text"""
    req_image = []
    text_list = []
    image_pdf = WImage(filename=file,
                 resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    
    for img in image_jpeg.sequence:
        img_page = WImage(image=img)
        req_image.append(img_page.make_blob('jpeg'))

    for img in req_image:
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        text_list.append(txt)
    return text_list
        

In [ ]:
pdf_dir = path
pdfs = [{'file': file, 'texts': extract_text(str(Path.joinpath(path, file)))} for file in os.listdir(pdf_dir) if file.endswith('.pdf')]

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML('\n'.join(
#    f"<h1>{pdf['file']}</h1>" +
#    ''.join(f"<h2>Page {i + 1}</h2><p>{p}</p>".replace('\n', '<br>') for i, p in enumerate(pdf['texts']))
#    for pdf in pdfs
#)))

In [ ]:
for pdf in pdfs:
    with transaction.atomic():
        doc = OcrDocument(filename=pdf['file'])
        doc.save()
        OcrPage.objects.bulk_create([OcrPage(page_num=i+1, text=text, ocr_document=doc) for i, text in enumerate(pdf['texts'])])